https://code.visualstudio.com/docs/datascience/data-science-tutorial#_optional-use-a-neural-network

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('titanic3.csv')

In [2]:
df.replace('?', np.nan, inplace= True)
df = df.astype({"age": np.float64, "fare": np.float64})

In [3]:
df.replace({'male': 1, 'female': 0}, inplace=True)


/var/folders/l4/vd99d1jj46l4pg5l2_852z280000gn/T/ipykernel_10160/3191515041.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'male': 1, 'female': 0}, inplace=True)


In [4]:
df['relatives'] = df.apply (lambda row: int((row['sibsp'] + row['parch']) > 0), axis=1)
df.corr(numeric_only=True).abs()[["survived"]]


,survived
pclass,0.312469
survived,1.000000
sex,0.528693
age,0.055512
sibsp,0.027825
parch,0.082660
fare,0.244265
body,NaN
relatives,0.201719


In [5]:
df = df[['sex', 'pclass','age','relatives','fare','survived']].dropna()


In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df[['sex','pclass','age','relatives','fare']], df.survived, test_size=0.2, random_state=0)


In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(x_train)
X_test = sc.transform(x_test)


In [8]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

1) The first layer will be set to have a dimension of 5, since you have five inputs: sex, pclass, age, and fare.
3) The last layer must output 1, since you want a 1-dimensional output indicating whether a passenger would survive.
2) The middle layer was kept at 5 for simplicity, although that value could have been different.

The rectified linear unit (relu) activation function is used as a good general activation function for the first two layers, while the sigmoid activation function is required for the final layer as the output you want (of whether a passenger survives or not) needs to be scaled in the range of 0-1 (the probability of a passenger surviving).

In [9]:
model.add(Dense(5, kernel_initializer = 'uniform', activation = 'relu', input_dim = 5))
model.add(Dense(5, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(1, kernel_initializer = 'uniform', activation = 'sigmoid'))


/opt/anaconda3/envs/apple_ml/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 5)              │            30 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │            30 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 66 (264.00 B)

 Trainable params: 66 (264.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=50)


Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5435 - loss: 0.6928   
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5886 - loss: 0.6909 
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5927 - loss: 0.6882 
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5933 - loss: 0.6837 
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5784 - loss: 0.6773 
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step - accuracy: 0.6168 - loss: 0.6653
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 977us/step - accuracy: 0.7372 - loss: 0.6512
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7547 - loss: 0.6304 
Epoch 9/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7775 - loss: 0.6046 
Epoch 10/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 951us/step - accuracy: 0.7508 - loss: 0.5822
Epoch 11/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 955us/step - accuracy: 0.7894 - loss: 0.5536
Epoch 12/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accu

In [12]:
from sklearn import metrics
y_pred = np.rint(model.predict(X_test).flatten())
print(metrics.accuracy_score(y_test, y_pred))


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
0.7990430622009569
